In [40]:
# 08 - Logistic Regression
## 1 Decide our model (input, output, forward pass: operations, layers)
## 2 Construct loss and optimizer 
## 3 Training loop:
# - forward pass: compute predciton 
# - backward pass: compute gradients
# - update weights

import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

# 0) preparation of data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_feature = X.shape

#print(n_samples, n_feature)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

#scale

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

## convert label value data to a column data 
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


# 1) model
#f = wx + b, sigmoid function at the end

class LogisticRegression(nn.Module):
    def __init__(self, n__input_features, n_label):
        super().__init__()
        self.ln = nn.Linear(in_features=n__input_features, out_features=n_label)
        nn.init.normal_(self.ln.weight, mean=0, std=0.1)
        nn.init.constant_(self.ln.bias, 0)

    def forward(self, x):
        # sigmoid function will turn a value to close 0 or 1
        y_predicted = torch.sigmoid(self.ln(x))
        return y_predicted
    
model = LogisticRegression(n_feature, 1)

# 2) loss and optimizer
learning_rate = 0.01 
creterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) training loop

num_epoch = 100

for epoch in range(num_epoch):
    #forward
    y_pred = model(X_train)
    loss = creterion(y_pred, y_train)

    #backward dl/dw cacluate gradient of loss function 
    loss.backward()

    #update
    optimizer.step()

    #zero gradient 
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

## evaluation 
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / y_test.shape[0]
    print(f'accurary = {acc.item():.4f}')

epoch: 10, loss = 0.4930
epoch: 20, loss = 0.4242
epoch: 30, loss = 0.3774
epoch: 40, loss = 0.3433
epoch: 50, loss = 0.3171
epoch: 60, loss = 0.2962
epoch: 70, loss = 0.2789
epoch: 80, loss = 0.2644
epoch: 90, loss = 0.2520
epoch: 100, loss = 0.2412
accurary = 0.9035


---
accurary = 0.8947
Accuracy is not that high, so other parameters should be turned to achive higher accuracy